# Setup

In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=d1ecb3ab954da35702c0eaa37862fbce09de61ca155ea5bb8a92c9a5429e34d5
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import numpy as np
import pandas_ta as ta
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Data Prep

In [ ]:
# Get data to test on
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/SOLUSDT-5m-2022-2023-labeled")
df.index = pd.to_datetime(df["Date"])
df.drop(columns = ["Date"],inplace = True)
df.sort_index(inplace = True)

# Feature Extraction

## Lines

In [ ]:
OHLC = ["Open","Close","High","Low"]
def add_distances(feature,df):
   for price in OHLC:
        df[f"Absolute_{price}-{feature}"] = df[price] - df[f"Absolute_{feature}"]
        df[f"RelativeCL_{price}-{feature}"] = df[f"Absolute_{price}-{feature}"] / df["Close"]
        df[f"RelativeATR10_{price}-{feature}"] = df[f"Absolute_{price}-{feature}"] / df[f'Absolute_ATR_10']
        df[f"RelativeATR20_{price}-{feature}"] = df[f"Absolute_{price}-{feature}"] / df[f'Absolute_ATR_20']
def add_relative_features(feature,p,df):
  df[f'RelativeCL_{feature}'] = df[f'Absolute_{feature}'] / df["Close"]
  df[f'RelativeATR{p}_{feature}'] = df[f'Absolute_{feature}'] / df[f'Absolute_ATR_{p}']

In [ ]:
periods = [3,5,10,15,20,50,100,200]
def add_moving_averages(df):
    for p in periods:
        df[f'Absolute_MA_Close_{p}'] = df['Close'].rolling(window=p).mean()
        weights = np.arange(1, p+1)
        df[f'Absolute_WMA_Close_{p}'] = df['Close'].rolling(window=p).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
        df[f'Absolute_EMA_Close_{p}'] = df['Close'].ewm(span=p, adjust=False).mean()
        ema2 = df[f'Absolute_EMA_Close_{p}'].ewm(span=5, adjust=False).mean()
        ema3 = ema2.ewm(span=5, adjust=False).mean()
        df[f'Absolute_TEMA_Close_{p}'] = 3 * df[f'Absolute_EMA_Close_{p}'] - 3 * ema2 + ema3
        moving_averages = [f"MA_Close_{p}",f"WMA_Close_{p}",f"EMA_Close_{p}",f"TEMA_Close_{p}"]
        for ma in moving_averages:
          add_relative_features(ma,p,df)
          add_distances(ma, df)
    return df

def add_ATR(df):
   for p in periods:
      df[f'Absolute_ATR_{p}'] = ta.atr(df['High'], df['Low'], df['Close'],length = p)
      df[f'Relative_ATR_{p}'] = df[f'Absolute_ATR_{p}'] / df["Close"]
   return df

In [ ]:
def preprocess_data(df):
    df = df.dropna()
    df = df.drop_duplicates()
    if isinstance(df.index, pd.DatetimeIndex):
        df = df.sort_index()
    df = df[df['Close'] != 0]
    df = df[df['Open'] != 0]
    return df

In [ ]:
def extract_features(df):
   df = add_ATR(df)
   df = add_moving_averages(df)
   df = preprocess_data(df)
   for c in OHLC:
       df.rename(columns = {c : ("Absolute_" + c)},inplace = True)
   df.rename(columns = {"Volume" : ("Relative_Volume")},inplace = True)
   return df

In [ ]:
df = extract_features(df)

In [ ]:
df.to_csv("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/SOLUSDT-5m-2022-2023-extracted")

In [ ]:
target_columns = ["Reversal","Potential_profit"]
target = df.loc[:,target_columns]
df.drop(columns=target_columns, inplace=True)
df[target_columns] = target

# Modeling

In [ ]:
# Get data to test on
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/SOLUSDT-5m-2022-2023-extracted")
df.index = pd.to_datetime(df["Date"])
df.drop(columns = ["Date"],inplace = True)
df.sort_index(inplace = True)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df["Reversal"][df["Reversal"].isin([1,2,3])] = 0
df["Reversal"][df["Reversal"] == 4] = 1
df["Reversal"][df["Reversal"] == 5] = 2
df["Reversal"][df["Reversal"] == 6] = 3

In [ ]:
df["Reversal"].unique()

array([2, 0, 1, 3])

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

# X = df.drop(['Potential_profit','Reversal'], axis=1)
# y = df["Reversal"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,shuffle = False)

# undersample = RandomUnderSampler(sampling_strategy="auto")  # Resample the majority class
# X_resampled, y_resampled = undersample.fit_resample(X_train, y_train)

# # New class distribution
# print("New class distribution:")
# print(y_resampled.value_counts())

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assuming y_train contains your training labels
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)

class_weight_dict = dict(zip(classes, class_weights))
print("Class weights:", class_weight_dict)
train_weights = np.vectorize(class_weight_dict.get)(y_train)

Class weights: {0: 0.27278867268407236, 1: 6.622855701311806, 2: 6.622855701311806, 3: 31.080899763220206}


In [ ]:
import xgboost as xgb
import numpy as np
# Setting parameters for XGBoost model
params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': 4,  # Number of classes
    'max_depth': None,  # Maximum depth of the tree
    'eta': 0.01,  # Learning rate
    'gamma': 0.1,  # Minimum loss reduction required to make a further partition
    'subsample': 0.9,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'eval_metric': 'merror',
    'tree_method' : "hist", 'device' : "cuda"# Evaluation metric to use
}

class_weight_dict = {0: 1, 1: 3, 2: 3, 3: 1}
train_weights = np.vectorize(class_weight_dict.get)(y_train)

dtrain = xgb.DMatrix(X_train, label=y_train, weight=train_weights)
dtest = xgb.DMatrix(X_test, label=y_test)

# Training the model
num_rounds = 5
evals = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain, num_rounds, evals, verbose_eval=True)

# Making predictions
y_pred = model.predict(dtest)

# Calculating accuracy
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [06:29:59] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


[0]	train-merror:0.18622	eval-merror:0.10260
[1]	train-merror:0.18467	eval-merror:0.09993
[2]	train-merror:0.18462	eval-merror:0.09862
[3]	train-merror:0.18480	eval-merror:0.09862
[4]	train-merror:0.18473	eval-merror:0.09904
Accuracy: 0.9009636993867367


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(dtest)

report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.93      0.97      0.95     48052
           1       0.36      0.18      0.24      2029
           2       0.31      0.21      0.25      2030
           3       0.00      0.00      0.00       395

    accuracy                           0.90     52506
   macro avg       0.40      0.34      0.36     52506
weighted avg       0.88      0.90      0.89     52506



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Strategy

In [ ]:
def get_strategy_actions(signals,Open,tp = 0.5, sl = 0.1):
    index = Open.index
    position = None
    entry_price = None
    buy = []
    sell = []
    exBuy_signal = []
    exSell_signal = []
    exBuy_pl = {"index":[],"price": []}
    exSell_pl = {"index":[],"price": []}
    for i in range(len(signals)):
       if position:
         if position == "Buy":
            sl_price = entry_price * (1-sl)
            tp_price = entry_price * (1+tp)
            if df.Absolute_Low[i] < sl_price:
              exBuy_pl["index"].append(index[i])
              exBuy_pl["price"].append(sl_price)
            if signals[i] in [2,3,5,6]:
              position = None
              exBuy_signal.append(index[i])
         else:
            if signals[i] in [1,3,4,6]:
              position = None
              exSell_signal.append(index[i])
       else:
         if signals[i] in [1,4]:
            position = "Buy"
            buy.append(index[i])
            entry_price = Open[i+1]
         elif signals[i] in [2,5]:
            position = "Sell"
            sell.append(index[i])
            entry_price = Open[i+1]
    sell = sell[:len(exSell_signal)]
    buy = buy[:len(exBuy_signal)]
    strategy_actions = {"buy": buy , "exBuy": exBuy_signal, "sell": sell, "exSell": exSell_signal}
    return strategy_actions

# Evaluation
Main Question: What is the estimated daily returns in the future?

## Functions

In [ ]:
def get_positions(strategy_actions, Open):
    def helper(arr1, arr2):
      if len(arr1) == 0:
          return np.array([], dtype=float)
      return np.vectorize(lambda x: x.total_seconds(), otypes='f')(np.array(arr1) - np.array(arr2))
    buy, exBuy, sell, exSell = strategy_actions["buy"], strategy_actions["exBuy"], strategy_actions["sell"], strategy_actions["exSell"]
    buy_positions = pd.DataFrame({"entry_price" : Open.shift(-1)[buy].values , "exit_price" : Open.shift(-1)[exBuy].values, "exit_date": exBuy, "buy_or_sell" : 1, "duration": helper(np.array(exBuy), np.array(buy))},index = buy)
    sell_positions = pd.DataFrame({"entry_price" : Open.shift(-1)[sell].values ,"exit_price" : Open.shift(-1)[exSell].values, "exit_date": exSell, "buy_or_sell" : -1, "duration": helper(np.array(exSell), np.array(sell))}, index = sell)
    positions = pd.concat([buy_positions,sell_positions]).sort_index()
    fees = (positions.entry_price * 0.001).values + (positions.exit_price * 0.001).values
    positions["return"] = (positions.exit_price - positions.entry_price - fees) / positions.entry_price * positions.buy_or_sell * 100
    return positions

def get_daily_returns(positions):
    return positions["return"].resample('D').sum()

def get_num_daily_trades(positions):
    return positions["return"].resample('D').count()

def plot_prob_dstrb(series):
  sns.displot(data=series,stat='probability')


def plot_mean_deviation(series):
  mean_value = series.mean()
  plt.figure(figsize=(10, 6))
  plt.scatter(series.reset_index().index, series, color='blue', label='Data Points')
  plt.axhline(y=mean_value, color='red', linestyle='-', linewidth=2, label=f'Mean: {mean_value:.2f}')
  for i, value in enumerate(series):
      plt.plot([i, i], [value, mean_value], color='gray', linestyle='--', linewidth=0.5)
  plt.title('Mean Deviation Plot')
  plt.xlabel('Index')
  plt.ylabel('Value')
  plt.legend()
  plt.grid(True)
  plt.show()




In [ ]:
def evaluate(true_signals_train,true_signals_test, predicted_signals_train, predicted_signals_test ,
             true_pp_train, true_pp_test, predicted_pp_train, predicted_pp_test,
             strategy_actions, df_org):
  df = df_org.copy()
  org_index = df.index
  Open = df.Absolute_Open
  print("Signal Evaluation \n")
  print("Train Data\n")
  # target_names = ['non','buy', 'sell', 'std','Mbuy','Msell','Mstd']
  target_names = ['non','buy', 'sell', 'std']
  print(classification_report(true_signals_train, predicted_signals_train, target_names=target_names))
  cm = confusion_matrix(true_signals_train, predicted_signals_train)
  ConfusionMatrixDisplay(cm).plot()
  print("Test Data\n")
  print(classification_report(true_signals_test, predicted_signals_test, target_names=target_names))
  cm = confusion_matrix(true_signals_test, predicted_signals_test)
  ConfusionMatrixDisplay(cm).plot()
  print()
  print("\n Potential Profit Evaluation \n")
  print(f"train mae: {mean_absolute_error(true_pp_train, predicted_pp_train)}")
  print(f"test mae: {mean_absolute_error(true_pp_test, predicted_pp_test)}")
  print("\n \n" + "-"*50 + "\n \n")

  print("Strategy Evaluation \n")
  print(f"Backtest Duration: {Open.index[-1] - Open.index[0]} From {Open.index[0]} to {Open.index[-1]}\n")
  print("Trade Returns Statistics \n")
  predicted_positions = get_positions(strategy_actions,Open)
  plot_prob_dstrb(predicted_positions["return"])
  plot_mean_deviation(predicted_positions["return"])
  print("Trade Duration Statistics \n")
  plot_prob_dstrb(predicted_positions["duration"])
  plot_mean_deviation(predicted_positions["duration"])
  print("Daily Returns Statistics \n")
  daily_returns = get_daily_returns(predicted_positions)
  plot_prob_dstrb(daily_returns)
  plot_mean_deviation(daily_returns)
  print("Num Daily Trades Statistics\n")
  num_daily_trades = get_num_daily_trades(predicted_positions)
  plot_prob_dstrb(num_daily_trades)
  plot_mean_deviation(num_daily_trades)
  num_winning_trades = len(predicted_positions["return"][predicted_positions["return"] > 0])
  win_rate = num_winning_trades / len(predicted_positions["return"])
  print(f"Win Rate: {win_rate}")

In [ ]:
def plot_eval(true_signals,predicted_signals,true_pp,predicted_pp,positions,df):
  true_signals = pd.Series(true_signals)
  true_signals = true_signals.reset_index(drop = True)
  predicted_signals = pd.Series(predicted_signals)
  predicted_signals = predicted_signals.reset_index(drop = True)
  df_index = df.index
  df = df.reset_index(drop = True)
  fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Absolute_Open'],
                high=df['Absolute_High'],
                low=df['Absolute_Low'],
                close=df['Absolute_Close'])])
  # True signals
  # fig.add_trace(go.Scatter(x=df.index[true_signals == 1], y=df.Absolute_Low[true_signals == 1], mode='markers',marker =dict(color='green',size = 5,symbol='star') ))
  # fig.add_trace(go.Scatter(x=df.index[true_signals == 2], y=df.Absolute_High[true_signals == 2], mode='markers',marker =dict(color='red',size = 5,symbol='star') ))
  # fig.add_trace(go.Scatter(x=df.index[true_signals == 3], y=df.Absolute_High[true_signals == 3], mode='markers',marker =dict(color='yellow',size = 5,symbol='star') ))
  # fig.add_trace(go.Scatter(x=df.index[true_signals == 4], y=df.Absolute_Low[true_signals == 4], mode='markers',marker =dict(color='green',size = 7,symbol='star') ))
  # fig.add_trace(go.Scatter(x=df.index[true_signals == 5], y=df.Absolute_High[true_signals == 5], mode='markers',marker =dict(color='red',size = 7,symbol='star') ))
  # fig.add_trace(go.Scatter(x=df.index[true_signals == 6], y=df.Absolute_High[true_signals == 6], mode='markers',marker =dict(color='yellow',size = 7,symbol='star') ))
  # Predicted Signals
  fig.add_trace(go.Scatter(x=df.index[predicted_signals == 1], y=df.Absolute_Low[predicted_signals == 1], mode='markers',marker =dict(color='green',size = 5,symbol='arrow') ))
  fig.add_trace(go.Scatter(x=df.index[predicted_signals == 2], y=df.Absolute_High[predicted_signals == 2], mode='markers',marker =dict(color='red',size = 5,symbol='arrow') ))
  fig.add_trace(go.Scatter(x=df.index[predicted_signals == 3], y=df.Absolute_High[predicted_signals == 3], mode='markers',marker =dict(color='yellow',size = 5,symbol='arrow') ))
  fig.add_trace(go.Scatter(x=df.index[predicted_signals == 4], y=df.Absolute_Low[predicted_signals == 4], mode='markers',marker =dict(color='green',size = 7,symbol='arrow') ))
  fig.add_trace(go.Scatter(x=df.index[predicted_signals == 5], y=df.Absolute_High[predicted_signals == 5], mode='markers',marker =dict(color='red',size = 7,symbol='arrow') ))
  fig.add_trace(go.Scatter(x=df.index[predicted_signals == 6], y=df.Absolute_High[predicted_signals == 6], mode='markers',marker =dict(color='yellow',size = 7,symbol='arrow') ))
  # Predicted and True Potential Profit
  true_pp = true_pp[df["Reversal"] != 0]
  predicted_pp = predicted_pp[df["Reversal"] != 0]
  # fig.add_trace(go.Scatter(x=df.index[df["Reversal"] != 0], y=df.Absolute_Low[df["Reversal"] != 0] - df.Absolute_Low.mean()*0.001, mode='text',text = true_pp.round(2),textfont=dict(size=4, color = "green")) )
  # fig.add_trace(go.Scatter(x=df.index[df["Reversal"] != 0], y=df.Absolute_Low[df["Reversal"] != 0] - df.Absolute_Low.mean()*0.005, mode='text',text = predicted_pp.round(2),textfont=dict(size=4, color = "blue")) )
  # Positions
  for (entry_date, entry_price, exit_date, exit_price, buy) in zip(positions.index, positions['entry_price'], positions['exit_date'], positions['exit_price'], positions['buy_or_sell']):
    color = "green" if buy == 1 else "red"
    fig.add_trace(go.Scatter(x=[df_index.get_loc(entry_date), df_index.get_loc(exit_date)],
                             y=[entry_price, exit_price],
                             mode='lines',
                             line = dict(color = color)))
  fig.show()


## Execution

In [ ]:
# class_train_pred = model.predict(X_train)
class_test_pred = model.predict(X_test)
# class_train_pred = np.argmax(class_train_pred, axis=1)
class_test_pred = np.argmax(class_test_pred, axis=1)
# reg_train_pred = reg_train_pred.flatten()
# reg_test_pred = reg_test_pred.flatten()

985/985 [==============================] - 4s 4ms/step


In [ ]:
class_test_pred = model.predict(dtest)
class_test_pred = np.argmax(class_test_pred, axis=1)

class_train_pred = model.predict(dtrain)
class_train_pred = np.argmax(class_train_pred, axis=1)

In [ ]:
np.unique(class_test_pred,return_counts=True)

(array([3]), array([31504]))

In [ ]:
!pip install backtesting

In [ ]:
df_test["Prediction"] = class_test_pred

In [ ]:
# change backtesting library: no plotting, reversal is not giving 100% winrate, and cannot compound and control when to keep money and when to draw it

from backtesting import Backtest, Strategy

class MLStrategy(Strategy):
    def init(self):
       pass

    def next(self):
        price = self.data.Close[-1]
        signal = self.data.Prediction[-1]
        sl = 0.005
        tp = 0.005
        if signal == 1:
            if self.position.is_short:
                self.position.close()
            else:
                self.buy(sl=price * (1-sl), tp=price * (1+tp))
        elif signal == 2:
            if self.position.is_long:
                self.position.close()
            else:
              self.sell(sl=price * (1+sl), tp=price * (1-tp))
        elif signal == 3:
           self.position.close()


test = df_test.rename(columns = {"Absolute_Open" : "Open", "Absolute_Close" : "Close", "Absolute_High" : "High", "Absolute_Low" : "Low", "Relative_Volume" : "Volume"})
bt = Backtest(test, MLStrategy, cash=100000, commission = 0.001)
output = bt.run()
output
# bt.plot()

Start                     2023-09-13 14:40:00
End                       2023-12-31 23:55:00
Duration                    109 days 09:15:00
Exposure Time [%]                    9.890808
Equity Final [$]                  56234.71061
Equity Peak [$]                  104692.02874
Return [%]                         -43.765289
Buy & Hold Return [%]              458.287596
Return (Ann.) [%]                  -85.262951
Volatility (Ann.) [%]                3.622398
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -46.639271
Avg. Drawdown [%]                   -2.349297
Max. Drawdown Duration       96 days 04:55:00
Avg. Drawdown Duration        4 days 08:16:00
# Trades                                  613
Win Rate [%]                        49.755302
Best Trade [%]                       0.452904
Worst Trade [%]                     -0.652806
Avg. Trade [%]                    

In [ ]:
bt.plot()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
strategy_actions = get_strategy_actions(class_test_pred,df_test.Absolute_Open)

In [ ]:
# evaluate(y_class_train,y_class_test, class_train_pred, class_test_pred ,
#              y_reg_train, y_reg_test, reg_train_pred, reg_test_pred,
#              strategy_actions, df_test)

evaluate(y_class_train,y_class_test, class_train_pred, class_test_pred ,
             y_reg_train, y_reg_test, y_reg_train, y_reg_test,
             strategy_actions, df_test)

In [ ]:
strategy_actions

In [ ]:
window = 2000
strategy_actions = get_strategy_actions(class_test_pred[:window],df_test[:window].Absolute_Open)
positions = get_positions(strategy_actions, df_test[:window].Absolute_Open)
# plot_eval(y_class_test[:window],class_test_pred[:window],y_reg_test[:window],reg_test_pred[:window],positions,df_test[:window])
plot_eval(y_class_test[:window],class_test_pred[:window],y_reg_test[:window],y_reg_test[:window],positions,df_test[:window])

In [26]:
import sys
sys.path.append('drive/MyDrive/Colab Notebooks/Trading Bot/Utils')

import fn

In [ ]:
fn.